In [16]:
import random

In [11]:
from firebase_admin import initialize_app, credentials, db

In [2]:
cred = credentials.Certificate('creds.json')

In [3]:
app = initialize_app(cred, {'databaseURL': "https://calzan-default-rtdb.europe-west1.firebasedatabase.app/"})

# Init

In [7]:
passwords = ['boss', 'rona', 'moin']

In [9]:
players = {}

for num, password in enumerate(passwords):
    
    ref = db.reference('/' + password)
    players[str(num+1)](ref)
    ref.set({
        "number": num,
        "resources": {"H": 0, "L": 0, "W": 0, "S": 0, "E": 0},
        "cards": {
            "knights": 0,
            "playedKnights": 0,
            "victory": [],
            "actions": [],
            "maxRoads": False,
            "maxKnights": False
        }
    })

In [5]:
board = [    
    "                                 x           ",
    "                              + = +          ",
    "                            /     |          ",
    "                   e  + - +   W   + - +      ",
    "                    %     |   4 /     $ s    ",
    "              + - +   L   + - +   H   + - +  ",
    "            /     |   2 /     |   6 /     |  ",
    "          +   S   + - +   H   + - +   W   +  ",
    "          |   9 /     |   5 /     |   9 /    ",
    "       w  + - +   L   + - +   S   + - +      ",
    "        %     |   8 /  *  |  12 /     $ x    ",
    "      +   H   + - +   X   + - +   S   +      ",
    "      |   3 /     |     /     |   4 /        ",
    "      + - +   E   + - +   E   + - +          ",
    "    /     |   5 /     |   8 /     |          ",
    "  +   W   + - +   L   + - +   S   +          ",
    "x $   3 /     |  10 /     |  10 %            ",
    "  + - +   W   + - +   H   + - +  x           ",
    "      |   6 /     |  11 /                    ",
    "      + = +   E   + = +                      ",
    "       h  |  11 /  l                         ",
    "          + - +                              ",
    "                                             "
]

In [173]:
db.reference("/board").set(board)

In [23]:
stack = ["Ritter"] * 14 + ["Monopol", "Strassenbau", "Erfindung"] * 2 + ["Bibliothek", "Marktplatz", "Rathaus", "Kirche", "Universität"]

In [28]:
random.shuffle(stack)

In [101]:
stack.pop(0)

'Ritter'

# Würfeln

In [146]:
numbers = list(players)
nxt = 0

In [171]:
roll = random.randint(1, 6) + random.randint(1, 6)
nxt = (nxt + 1) % 3 
db.reference("/turn").set({'player': numbers[nxt], 'roll': str(roll)})

In [ ]:
rand_res = roll = random.randint(1, 6)

# Game

In [172]:
board = [    
    "                                 x           ",
    "                              + = +          ",
    "                            /     |          ",
    "                   e  + -2+   W   + - +      ",
    "                    %     |2  4 /     $ s    ",
    "              + - +   L   + -2#2  H   + - +  ",
    "            /     |   2 /     |   6 /     |  ",
    "          +   S   + -0+0  H   + - +1  W   +  ",
    "          |   9 /     |   5 /     |1  9 /    ",
    "       w  + - +   L   + - +   S   + -1+1     ",
    "        %     |   8 /     |  12 /1    $ x    ",
    "      +   H   +0- +   X   + - +1  S   +      ",
    "      |   3 /0    |     /     |1  4 /        ",
    "      + - +   E   + - +   E   + - +          ",
    "    /     |0  5 /     |   8 /1    |          ",
    "  +   W   + - +2  L   #1- +   S   +          ",
    "x $   3 /0 *  |2 10 /1    |  10 %            ",
    "  + - +0  W   + - +   H   + - +  x           ",
    "      |   6 /2    |  11 /                    ",
    "      + = +2  E   + = +                      ",
    "       h  |  11 /  l                         ",
    "          + - +                              ",
    "                                             "
]

In [173]:
db.reference("/board").set(board)